In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import Session
from test_model import Person


In [2]:
engine = create_engine('sqlite:///test_db', echo=True)
session = Session(engine)
persons = session.query(Person).all()
for person in persons:
    print(f'id:{person.id} name:{person.name} size:{person.size}')


2024-06-22 15:56:38,012 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 15:56:38,072 INFO sqlalchemy.engine.Engine SELECT person.id AS person_id, person.name AS person_name, person.size AS person_size 
FROM person
2024-06-22 15:56:38,087 INFO sqlalchemy.engine.Engine [generated in 0.01572s] ()
id:1 name:name_a size:10.5
id:2 name:name_b size:None
id:3 name:c_update size:3.14
id:100 name:name_z size:99.9


In [4]:
new_persion = Person(id=4, name='from_model', size=100)
session.add(new_persion)
session.commit()
persons = session.query(Person).all()
for person in persons:
    print(person.__dict__)


2024-06-22 15:57:16,925 INFO sqlalchemy.engine.Engine INSERT INTO person (id, name, size) VALUES (?, ?, ?)
2024-06-22 15:57:16,926 INFO sqlalchemy.engine.Engine [generated in 0.00119s] (4, 'from_model', 100.0)
2024-06-22 15:57:16,931 INFO sqlalchemy.engine.Engine COMMIT
2024-06-22 15:57:16,967 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 15:57:16,970 INFO sqlalchemy.engine.Engine SELECT person.id AS person_id, person.name AS person_name, person.size AS person_size 
FROM person
2024-06-22 15:57:16,972 INFO sqlalchemy.engine.Engine [cached since 38.9s ago] ()
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f1856ef1750>, 'id': 1, 'name': 'name_a', 'size': 10.5}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f1856ef1600>, 'id': 2, 'name': 'name_b', 'size': None}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f1856ef1810>, 'id': 3, 'name': 'c_update', 'size': 3.14}
{'_sa_instance_state': <sqlalchemy.orm.s

In [5]:
person4 = session.query(Person).filter(Person.id==4).first()
person4.size = 2.236
session.commit()
session.query(Person).filter(Person.size>10).delete()
session.commit()



2024-06-22 15:57:50,553 INFO sqlalchemy.engine.Engine SELECT person.id AS person_id, person.name AS person_name, person.size AS person_size 
FROM person 
WHERE person.id = ?
 LIMIT ? OFFSET ?
2024-06-22 15:57:50,555 INFO sqlalchemy.engine.Engine [generated in 0.00171s] (4, 1, 0)
2024-06-22 15:57:50,558 INFO sqlalchemy.engine.Engine UPDATE person SET size=? WHERE person.id = ?
2024-06-22 15:57:50,569 INFO sqlalchemy.engine.Engine [generated in 0.01092s] (2.236, 4)
2024-06-22 15:57:50,571 INFO sqlalchemy.engine.Engine COMMIT
2024-06-22 15:57:50,716 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 15:57:50,719 INFO sqlalchemy.engine.Engine DELETE FROM person WHERE person.size > ?
2024-06-22 15:57:50,720 INFO sqlalchemy.engine.Engine [generated in 0.00147s] (10,)
2024-06-22 15:57:50,722 INFO sqlalchemy.engine.Engine COMMIT


In [6]:
result_list = session.execute('SELECT * FROM person WHERE id = :id', {"id":4}) 
for result in result_list:
    print(result)

session.close()



2024-06-22 15:58:14,564 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 15:58:14,565 INFO sqlalchemy.engine.Engine SELECT * FROM person WHERE id = ?
2024-06-22 15:58:14,566 INFO sqlalchemy.engine.Engine [generated in 0.00101s] (4,)
(4, 'from_model', 2.236)
2024-06-22 15:58:14,577 INFO sqlalchemy.engine.Engine ROLLBACK


In [7]:
from test_model import Human
from sqlalchemy import create_engine, update, delete
from sqlalchemy.orm import sessionmaker
from sqlalchemy.exc import SQLAlchemyError
engine = create_engine('sqlite:///test_db', echo=True)
Session = sessionmaker(bind=engine)
session = Session()

In [8]:
try:
    humans = session.query(Human).all()
    for human in humans:
        print(f"ID: {human.id}, Name: {human.name}, Height: {human.height}, Weight: {human.weight}")

except SQLAlchemyError as e:
    print(f"SQLAlchemyエラー: {e}")

finally:
    session.close()

2024-06-22 16:26:05,553 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 16:26:05,556 INFO sqlalchemy.engine.Engine SELECT human.id AS human_id, human.name AS human_name, human.height AS human_height, human.weight AS human_weight 
FROM human
2024-06-22 16:26:05,559 INFO sqlalchemy.engine.Engine [generated in 0.00283s] ()
ID: 2, Name: Bob, Height: 180.2, Weight: None
ID: 3, Name: Carol, Height: None, Weight: 55.0
2024-06-22 16:26:05,566 INFO sqlalchemy.engine.Engine ROLLBACK


In [9]:
try:
    new_human = Human(name='New Person', height=170.5, weight=65.2)
    session.add(new_human)
    session.commit()
    print("新しいレコードを追加しました。")

except SQLAlchemyError as e:
    print(f"SQLAlchemyエラー: {e}")
    session.rollback()

finally:
    session.close()


2024-06-22 16:26:18,947 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 16:26:18,959 INFO sqlalchemy.engine.Engine INSERT INTO human (name, height, weight) VALUES (?, ?, ?)
2024-06-22 16:26:18,961 INFO sqlalchemy.engine.Engine [generated in 0.00123s] ('New Person', 170.5, 65.2)
2024-06-22 16:26:18,965 INFO sqlalchemy.engine.Engine COMMIT
新しいレコードを追加しました。


In [10]:
try:
    session.query(Human).filter(Human.height == 180.0).update({Human.name: 'Updated Name'}, synchronize_session='fetch')
    session.commit()
    print("heightが180.0のレコードを更新しました。")

except SQLAlchemyError as e:
    print(f"SQLAlchemyエラー: {e}")
    session.rollback()

finally:
    session.close()

2024-06-22 16:26:51,434 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 16:26:51,438 INFO sqlalchemy.engine.Engine SELECT human.id 
FROM human 
WHERE human.height = ?
2024-06-22 16:26:51,440 INFO sqlalchemy.engine.Engine [generated in 0.00196s] (180.0,)
2024-06-22 16:26:51,631 INFO sqlalchemy.engine.Engine UPDATE human SET name=? WHERE human.height = ?
2024-06-22 16:26:51,637 INFO sqlalchemy.engine.Engine [generated in 0.00585s] ('Updated Name', 180.0)
2024-06-22 16:26:51,650 INFO sqlalchemy.engine.Engine COMMIT
heightが180.0のレコードを更新しました。


In [11]:
try:
    session.query(Human).filter(Human.weight == 55.0).delete()
    session.commit()
    print("weightが55.0のレコードを削除しました。")

except SQLAlchemyError as e:
    print(f"SQLAlchemyエラー: {e}")
    session.rollback()

finally:
    session.close()

2024-06-22 16:26:53,581 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-06-22 16:26:53,584 INFO sqlalchemy.engine.Engine DELETE FROM human WHERE human.weight = ?
2024-06-22 16:26:53,585 INFO sqlalchemy.engine.Engine [generated in 0.00092s] (55.0,)
2024-06-22 16:26:53,587 INFO sqlalchemy.engine.Engine COMMIT
weightが55.0のレコードを削除しました。
